<a href="https://colab.research.google.com/github/Mickey1225/jupyter-notebook/blob/main/YOLO_v8_%5BRipe_and_Unripe_Tomatoes_Detection%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sumn2u_riped_and_unriped_tomato_dataset_path = kagglehub.dataset_download('sumn2u/riped-and-unriped-tomato-dataset')

print('Data source import complete.')


## 1. Data Preparation

The given dataset is provided in the following format:

```sh
Riped and Unriped Tomato Dataset/
├── Images/
│   ├── riped_tomato_1.jpeg
│   ├── riped_tomato_10.jpeg
│   ├── riped_tomato_11.jpeg
│   ├── riped_tomato_12.jpeg
│   └── ...
└── Labels/
    ├── riped_tomato_1.txt
    ├── riped_tomato_10.txt
    ├── riped_tomato_11.txt
    ├── riped_tomato_12.txt
    └── ...
```

The recommended format differs from what we have, so we will split the dataset to support the [YOLO fromat](https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format). For this, we will import some libraries.


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split # To split the data into train and test
import cv2
import random
from concurrent.futures import ThreadPoolExecutor
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import glob

In [ ]:
# Define base path for the dataset
base_data_dir = os.path.join("/kaggle","input", "riped-and-unriped-tomato-dataset")
print(f"Base Data Directory: {base_data_dir}")

In [ ]:
# Define specific paths for images and labels
images_dir = os.path.join(base_data_dir, "Riped and Unriped Tomato Dataset", "Images")
labels_dir = os.path.join(base_data_dir, "Riped and Unriped Tomato Dataset", "labels")

print(f"Images Dir: {images_dir}, Labels Dir: {labels_dir}")

Let's check the file extensions that we have in image directory

In [ ]:
# Get a list of all files in the directory
files = glob.glob(os.path.join(images_dir, '*'))

# Collect unique file extensions
extensions = set()
for file in files:
    _, ext = os.path.splitext(file)
    extensions.add(ext.lower())  # Convert to lowercase to handle case sensitivity

# Print all unique extensions found
print("Unique File Extensions:")
for ext in extensions:
    print(ext)

We have all JPEG files. The below function loads images and corresponding annotation paths from directories.

In [ ]:
def load_data(images_dir, labels_dir):
    images = []
    annotations = []

    for filename in os.listdir(images_dir):
        if filename.endswith('.jpeg'):
            image_path = os.path.join(images_dir, filename)
            annotation_path = os.path.join(labels_dir, filename.replace('.jpeg', '.txt'))

            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)

    return images, annotations

In [ ]:
images, annotations = load_data(images_dir, labels_dir)

To visualizes a random sample of images. Let's create a `visualize_samples` function.

In [ ]:
def visualize_samples(images, num_samples=5):
    fig, axs = plt.subplots(1, num_samples, figsize=(15, 5))

    for i in range(num_samples):
        image = Image.open(images[i])
        axs[i].imshow(image)
        axs[i].axis('off')
        axs[i].set_title(f'Sample {i+1}')

    plt.show()

In [ ]:
visualize_samples(images)

Let's calculate and visualize the average height and width of the images in the dataset.

In [ ]:
def calculate_average_size(images_dir):
    total_width = 0
    total_height = 0
    total_images = 0

    for filename in os.listdir(images_dir):
        if filename.endswith('.jpeg'):  # assuming images are in .jpeg format
            image_path = os.path.join(images_dir, filename)
            image = Image.open(image_path)
            width, height = image.size
            total_width += width
            total_height += height
            total_images += 1

    if total_images > 0:
        average_width = total_width / total_images
        average_height = total_height / total_images
        return average_width, average_height
    else:
        return None

In [ ]:
def plot_average_size(average_width, average_height):
    categories = ['Average Width', 'Average Height']
    values = [average_width, average_height]

    plt.figure(figsize=(6, 4))
    plt.bar(categories, values, color=['skyblue', 'lightgreen'])
    plt.xlabel('Categories')
    plt.ylabel('Pixels')
    plt.title('Average Image Dimensions')
    plt.ylim(0, max(values) * 1.1)
    plt.grid(axis='y')
    plt.show()

In [ ]:
average_width, average_height = calculate_average_size(images_dir)

if average_width and average_height:
    print(f"Average Image Width: {average_width:.2f} pixels")
    print(f"Average Image Height: {average_height:.2f} pixels")

    # Plot average size
    plot_average_size(average_width, average_height)
else:
    print("No images found or unable to calculate average size.")

We are also going to plot the distribution of classes ('ripe' and 'unripe') so, let's create a seperate function for this.

In [ ]:
def plot_class_distribution(annotations):
    classes = {'ripe': 0, 'unripe': 0}

    for annotation_file in annotations:
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_label = line.split()[0]  # assuming YOLO format with class label as first entry
                if class_label == '0':
                    classes['unripe'] += 1
                elif class_label == '1':
                    classes['ripe'] += 1
    # Print the class counts
    for cls, count in classes.items():
        print(f"{cls}: {count}")

    # Plotting
    plt.bar(classes.keys(), classes.values())
    plt.title('Class Distribution')
    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.show()

In [ ]:
plot_class_distribution(annotations)

Let's explore the RGB distributions of our datasets. Generally, the ripe tomatoes will have higher intensity in the red channel, whereas unripe tomatoes might have higher intensity in the green channel.

In [ ]:
from glob import glob

In [ ]:
def read_annotations(annotation_path):
    with open(annotation_path, 'r') as file:
        return file.read().strip()

In [ ]:
def read_image(img_path):
    img = cv2.imread(img_path)
    if img is not None:
        return img
    return np.array([])

In [ ]:
def process_images_in_batches(images, max_images=50, batch_size=10):
    """Process images in batches to reduce memory usage."""
    r_values, g_values, b_values = [], [], []

    # Limit the number of images to max_images
    images = images[:max_images]

    for i in range(0, len(images), batch_size):
        batch = images[i:i + batch_size]
        with ThreadPoolExecutor() as executor:
            img_list = list(executor.map(read_image, batch))

        for img in img_list:
            if img.size > 0:
                r_values.extend(img[:, :, 0].flatten())
                g_values.extend(img[:, :, 1].flatten())
                b_values.extend(img[:, :, 2].flatten())

    return r_values, g_values, b_values

def plot_rgb_distribution(images, category_name, max_images=50, batch_size=10):
    """Plot RGB distribution for the given images."""
    r_values, g_values, b_values = process_images_in_batches(images, max_images, batch_size)

    plt.figure(figsize=(15, 5))

    # Red Channel
    plt.subplot(1, 3, 1)
    r_hist, r_bins = np.histogram(r_values, bins=256, range=(0, 256))
    plt.bar(r_bins[:-1], r_hist, width=1, color='red', alpha=0.5)
    max_r_freq = r_hist.max()
    plt.axhline(max_r_freq, color='red', linestyle='--', label=f'Max Frequency: {max_r_freq}')
    plt.title(f'{category_name} - Red Channel')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Green Channel
    plt.subplot(1, 3, 2)
    g_hist, g_bins = np.histogram(g_values, bins=256, range=(0, 256))
    plt.bar(g_bins[:-1], g_hist, width=1, color='green', alpha=0.5)
    max_g_freq = g_hist.max()
    plt.axhline(max_g_freq, color='green', linestyle='--', label=f'Max Frequency: {max_g_freq}')
    plt.title(f'{category_name} - Green Channel')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Blue Channel
    plt.subplot(1, 3, 3)
    b_hist, b_bins = np.histogram(b_values, bins=256, range=(0, 256))
    plt.bar(b_bins[:-1], b_hist, width=1, color='blue', alpha=0.5)
    max_b_freq = b_hist.max()
    plt.axhline(max_b_freq, color='blue', linestyle='--', label=f'Max Frequency: {max_b_freq}')
    plt.title(f'{category_name} - Blue Channel')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Separate ripe and unripe tomatoes based on annotations
ripe_images = []
unripe_images = []

for img_path, annotation_path in zip(images, annotations):
    annotation = read_annotations(annotation_path)
    if annotation.startswith('1'):
        ripe_images.append(img_path)
    elif annotation.startswith('0'):
        unripe_images.append(img_path)

In [ ]:
# Debugging: Print the number of images in each category
print(f"Number of ripe images: {len(ripe_images)}")
print(f"Number of unripe images: {len(unripe_images)}")


In [ ]:
# Plot RGB distribution for ripe tomatoes
plot_rgb_distribution(ripe_images, 'Ripe Tomatoes')

In [ ]:
# Plot RGB distribution for unripe tomatoes
plot_rgb_distribution(unripe_images, 'Unripe Tomatoes')

From above, we can see that the **RED** color distribution is higher for ripe tomatoes whereas **Green** for unripe ones, indicating that ripe tomatoes have stronger presenence of red color in their images and green color for unripe ones. Lets apply various edge detection technique and see which better captures the tomato features.

* **Adaptive Thresholding**: Enhances contrast to highlight edges.
* **Laplacian Detection**: Detects rapid intensity changes.
* **Canny Detection**: Identifies detailed edges through gradient analysis.
* **Sobel Detection**: Finds edges using gradient in x and y directions.

In [ ]:
def calculate_edge_density(edges):
    return np.sum(edges) / edges.size

In [ ]:
def display_random_images_with_edges(basefolder, num_images=5):
    # Define image folder path and get list of image files
    image_folder = basefolder
    image_files = glob(os.path.join(image_folder, "*.jpeg"))

    # Check if there are enough images to display
    if len(image_files) < num_images:
        print(f"Not enough images in {image_folder} to display {num_images}.")
        return

    # Randomly select images
    selected_images = random.sample(image_files, num_images)

    # Create subplots to display the images
    fig, axes = plt.subplots(num_images, 5, figsize=(20, 4 * num_images))

    for idx, image_file in enumerate(selected_images):
        # Read and preprocess the image
        img_path = os.path.join(image_folder, image_file)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply various edge detection techniques
        # Adaptive Thresholding
        adaptive_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 3)

        # Laplacian Edge Detection
        lap = cv2.Laplacian(gray_img, cv2.CV_64F)
        lap = np.uint8(np.absolute(lap))
        lap_rgb = cv2.cvtColor(lap, cv2.COLOR_GRAY2RGB)

        # Canny Edge Detection
        mean_intensity = np.mean(gray_img)
        std_intensity = np.std(gray_img)
        lower_threshold = int(max(0, mean_intensity - 2 * std_intensity))
        upper_threshold = int(min(255, mean_intensity + 2 * std_intensity))
        edges = cv2.Canny(gray_img, lower_threshold, upper_threshold)

        # Sobel Edge Detection
        sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1)
        combined_sobel = cv2.bitwise_or(cv2.convertScaleAbs(sobelx), cv2.convertScaleAbs(sobely))
        combined_sobel_rgb = cv2.cvtColor(combined_sobel, cv2.COLOR_GRAY2RGB)

        # Display Original Image
        axes[idx, 0].imshow(img_rgb)
        axes[idx, 0].set_title('Original')
        axes[idx, 0].axis('off')

        # Display Adaptive Thresholded Image
        axes[idx, 1].imshow(adaptive_thresh, cmap='gray')
        axes[idx, 1].set_title('Adaptive Threshold')
        axes[idx, 1].axis('off')

        # Display Laplacian Edge Detection Image
        axes[idx, 2].imshow(lap_rgb)
        axes[idx, 2].set_title('Laplacian')
        axes[idx, 2].axis('off')

        # Display Canny Edge Detection Image
        axes[idx, 3].imshow(edges, cmap='gray')
        axes[idx, 3].set_title('Canny')
        axes[idx, 3].axis('off')

        # Display Sobel Edge Detection Image
        axes[idx, 4].imshow(combined_sobel_rgb)
        axes[idx, 4].set_title('Sobel')
        axes[idx, 4].axis('off')

    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()

In [ ]:
display_random_images_with_edges(images_dir, num_images=5)

Let's compare the edge density of above edge detection. For this, we will plot the frequency vs edge density of the above applied techniques.

In [ ]:
def display_edge_density(basefolder, num_images=5):
    image_files = glob(os.path.join(basefolder, "*.jpeg"))
    if len(image_files) < num_images:
        print(f"Not enough images in {basefolder} to display {num_images}.")
        return

    selected_images = random.sample(image_files, num_images)
    densities = {'Adaptive Threshold': [], 'Laplacian': [], 'Canny': [], 'Sobel': []}

    for img_file in selected_images:
        img_path = os.path.join(basefolder, img_file)
        img = cv2.imread(img_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Adaptive Thresholding
        adaptive_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 3)
        densities['Adaptive Threshold'].append(calculate_edge_density(adaptive_thresh))

        # Laplacian Edge Detection
        lap = cv2.Laplacian(gray_img, cv2.CV_64F)
        lap = np.uint8(np.absolute(lap))
        densities['Laplacian'].append(calculate_edge_density(lap))

        # Canny Edge Detection
        mean_intensity = np.mean(gray_img)
        std_intensity = np.std(gray_img)
        lower_threshold = int(max(0, mean_intensity - 2 * std_intensity))
        upper_threshold = int(min(255, mean_intensity + 2 * std_intensity))
        edges = cv2.Canny(gray_img, lower_threshold, upper_threshold)
        densities['Canny'].append(calculate_edge_density(edges))

        # Sobel Edge Detection
        sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1)
        combined_sobel = cv2.bitwise_or(cv2.convertScaleAbs(sobelx), cv2.convertScaleAbs(sobely))
        densities['Sobel'].append(calculate_edge_density(combined_sobel))

    # Plot the average edge densities for each method
    plt.figure(figsize=(12, 6))
    for method, density_values in densities.items():
        plt.hist(density_values, bins=10, alpha=0.5, label=method)

    plt.title('Edge Density Comparison')
    plt.xlabel('Edge Density')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

In [ ]:
display_edge_density(images_dir, num_images=5)

From the above, we can see the frequency and edge density of each technique. For detecting finer details or stronger edges, higher frequencies are better.

To count the number of images containing both 'ripe' and 'unripe' tomatoes let's create a count_mixed_images function that takes list of annotations as an arguments.

In [ ]:
def count_mixed_images(annotations):
    mixed_images_count = 0

    for annotation_file in annotations:
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
            has_ripe = False
            has_unripe = False
            for line in lines:
                class_label = int(line.split()[0])
                if class_label == 0:
                    has_unripe = True
                elif class_label == 1:
                    has_ripe = True
            if has_ripe and has_unripe:
                mixed_images_count += 1

    return mixed_images_count

In [ ]:
mixed_images_count = count_mixed_images(annotations)

print(f"Mixed Images Count: {mixed_images_count}")

To plot the count of images containing both 'ripe' and 'unripe' tomatoes. Let's have a seperate function.

In [ ]:
def plot_mixed_images_count(mixed_images_count):
    categories = ['Mixed Images']
    counts = [mixed_images_count]

    plt.figure(figsize=(6, 4))
    plt.bar(categories, counts, color='skyblue')
    plt.xlabel('Categories')
    plt.ylabel('Number of Images')
    plt.title('Images Containing Both Ripe and Unripe Tomatoes')
    plt.ylim(0, max(counts) * 1.1)
    plt.grid(axis='y')
    plt.show()

In [ ]:
# Plotting mixed images count
plot_mixed_images_count(mixed_images_count)

The dataset contains images that has both ripe and unripe tomatoes so, let's create a function to find and display sample images that has both.

In [ ]:
def show_sample_mixed_images(images_dir, labels_dir, num_samples=3):
    # Load dataset
    images, annotations = load_data(images_dir, labels_dir)

    # Initialize a list to store paths of mixed images
    mixed_image_paths = []

    # Iterate through annotations to find images with both classes
    for i, annotation_file in enumerate(annotations):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
            has_ripe = False
            has_unripe = False
            for line in lines:
                class_label = int(line.split()[0])  # assuming YOLO format with class label as first entry
                if class_label == 0:
                    has_unripe = True
                elif class_label == 1:
                    has_ripe = True
            if has_ripe and has_unripe:
                mixed_image_paths.append(images[i])

        # Stop when enough samples are found
        if len(mixed_image_paths) >= num_samples:
            break

    # Display sample images
    if mixed_image_paths:
        plt.figure(figsize=(15, 6))
        for i, image_path in enumerate(random.sample(mixed_image_paths, num_samples)):
            image = Image.open(image_path)
            plt.subplot(1, num_samples, i + 1)
            plt.imshow(image)
            plt.axis('off')
            plt.title(f'Sample {i+1}')
        plt.tight_layout()
        plt.show()
    else:
        print("No images found with both ripe and unripe tomatoes.")

In [ ]:
show_sample_mixed_images(images_dir, labels_dir, num_samples=5)

To draw the bounding boxes on an image based on annotations, we will be using draw_bounding_boxes function.

In [ ]:
def draw_bounding_boxes(image_path, annotation_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    with open(annotation_path, 'r') as f:
        for line in f:
            class_label, x_center, y_center, width, height = map(float, line.strip().split())
            # Convert YOLO format to bounding box coordinates
            img_width, img_height = image.size
            x_center *= img_width
            y_center *= img_height
            width *= img_width
            height *= img_height
            x_min = x_center - (width / 2)
            x_max = x_center + (width / 2)
            y_min = y_center - (height / 2)
            y_max = y_center + (height / 2)
            # Draw rectangle
            if class_label == 0:
                color = 'green'  # unripe
            else:
                color = 'red'  # ripe
            draw.rectangle([x_min, y_min, x_max, y_max], outline=color, width=2)
    return image

Since it's possible to have both ripe and unripe tomatoes in a single image, let's create a function to display these images with bounding boxes.

In [ ]:
def show_sample_mixed_images_with_bounding_boxes(images_dir, labels_dir, num_samples=3):
    # Load dataset
    images, annotations = load_data(images_dir, labels_dir)

    # Initialize a list to store paths of mixed images
    mixed_image_paths = []

    # Iterate through annotations to find images with both classes
    for i, annotation_file in enumerate(annotations):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
            has_ripe = False
            has_unripe = False
            for line in lines:
                class_label = int(line.split()[0])  # assuming YOLO format with class label as first entry
                if class_label == 0:
                    has_unripe = True
                elif class_label == 1:
                    has_ripe = True
            if has_ripe and has_unripe:
                mixed_image_paths.append((images[i], annotations[i]))

        # Stop when enough samples are found
        if len(mixed_image_paths) >= num_samples:
            break

    # Display sample images
    if mixed_image_paths:
        plt.figure(figsize=(15, 6))
        for i, (image_path, annotation_path) in enumerate(random.sample(mixed_image_paths, num_samples)):
            original_image = Image.open(image_path)
            annotated_image = draw_bounding_boxes(image_path, annotation_path)

            plt.subplot(2, num_samples, i + 1)
            plt.imshow(original_image)
            plt.axis('off')
            plt.title(f'Original {i+1}')

            plt.subplot(2, num_samples, num_samples + i + 1)
            plt.imshow(annotated_image)
            plt.axis('off')
            plt.title(f'Annotated {i+1}')

        plt.tight_layout()
        plt.show()
    else:
        print("No images found with both ripe and unripe tomatoes.")

In [ ]:
show_sample_mixed_images_with_bounding_boxes(images_dir, labels_dir, num_samples=3)

In [ ]:
# Handle potential train/test subdirectories
working_dir = os.path.join("/kaggle","working")
train_images_dir = os.path.join(working_dir, "train", "images")
train_labels_dir = os.path.join(working_dir, "train", 'labels')
test_images_dir = os.path.join(working_dir,"test", 'images')
test_labels_dir = os.path.join(working_dir, "test", 'labels')

print(f"Train Images Dir: {train_images_dir}, Train Labels Dir: {train_labels_dir}")
print(f"Test Images Dir: {test_images_dir}, Test Labels Dir: {test_labels_dir}")

In [ ]:
# Create train/test directories if they don't exist (using os.makedirs with exist_ok=True)
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

Now that we have created the respective folders, let's start importing the images.

In [ ]:
# Get list of images and labels
images = [f for f in os.listdir(images_dir) if f.endswith('.jpeg')]
labels = [f.replace('.jpeg', '.txt') for f in images]

Let's preview the number of images and labels we have.

In [ ]:
print(f"Images Length: {len(images)}")
print(f"Labels Length: {len(labels)}")

We have an equal number of images and labels. Now, we will split the dataset, where the training set will contain `80%` of the images and the remaining `20%` will be in the test set.

In [ ]:
# Split into training and testing sets
train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
train_labels = [f.replace('.jpeg', '.txt') for f in train_images]
test_labels = [f.replace('.jpeg', '.txt') for f in test_images]
print(f"Training set: {len(train_images)} images, {len(train_labels)} labels")
print(f"Testing set: {len(test_images)} images, {len(test_labels)} labels")

After splitting the dataset, let's move the files to their respective folders.

In [ ]:
# Copy files to the respective directories
def copy_files(file_list, src_path, dst_path):
    for file in file_list:
        shutil.copy(os.path.join(src_path, file), os.path.join(dst_path, file))

In [ ]:
copy_files(train_images, images_dir, train_images_dir)
copy_files(train_labels, labels_dir, train_labels_dir)
copy_files(test_images, images_dir, test_images_dir)
copy_files(test_labels, labels_dir, test_labels_dir)


In [ ]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2

    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area

In [ ]:

def calculate_iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

def convert_yolo_to_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    x_min = (x_center - width / 2) * img_width
    x_max = (x_center + width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    y_max = (y_center + height / 2) * img_height
    return [x_min, y_min, x_max, y_max]


ious = []

for annotation_file in annotations:
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        boxes = []
        for line in lines:
            _, x_center, y_center, width, height = map(float, line.split())
            bbox = convert_yolo_to_bbox([x_center, y_center, width, height], img_width=average_width, img_height=average_height)
            boxes.append(bbox)
        # Calculate IoU for every pair of boxes in the same image
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                iou = calculate_iou(boxes[i], boxes[j])
                ious.append(iou)

average_iou = np.mean(ious)
print("Average IoU:", average_iou)

## 2. Model Creation
For our project on detecting ripe and unripe tomatoes, let's install YOLO and verify its functionality.

In [ ]:
# setup
%pip install ultralytics
import ultralytics
ultralytics.checks()

Next, let's create a configuration file for YOLO. This file provides information about the dataset and the classes it contains.

In [ ]:
dataset_yaml = """
train: /kaggle/working/train/images
val: /kaggle/working/test/images

nc: 2  # number of classes
names: ['unripe', 'ripe']  # class names
"""

with open('/kaggle/working/dataset.yaml', 'w') as file:
    file.write(dataset_yaml)

Let's check the available commands in YOLOv8.

In [ ]:
!yolo

## 3. Model Training
We will train our model using `130` epochs with a batch size of `16`, and we will set the image size to `640`. The [exploratory data analysis (EDA)](https://www.kaggle.com/code/sumn2u/eda-of-ripe-and-unripe-tomatoes-dataset/notebook) shows more information about images and their sizes.

In [ ]:
from ultralytics import YOLO
from IPython.display import Image, display

# Load the trained YOLO model
model = YOLO('yolov8n.pt')
data_yaml_path = "/kaggle/working/dataset.yaml"

results = model.train(data = data_yaml_path,
            epochs=130,
            imgsz=720,
            device=0,
            lr0=0.01,  # initial learning rate
            lrf=0.001,  # final learning rate
            save_period=10
           )


## 4. Model Evaluation
Let's plot the testing results and then perform the evaluation on the validation set.

In [ ]:
from PIL import Image

In [ ]:
tpaths2=[]
for dirname, _, filenames in os.walk('/kaggle/working/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)

print(tpaths2[0])

In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()


In [ ]:
# evaluate model performance on the validation set
results = model.val()

## 5. Testing

Let's test our model with test images.

In [ ]:
# Define the path to the images
images_path = '/kaggle/working/test/images'
num_images = 5

# Sample a subset of images
images = random.sample([f for f in os.listdir(images_path) if f.endswith('.jpeg')], num_images)

# Show the results on test images
for image in images:
    img_path = os.path.join(images_path, image)
    results = model(img_path, conf=0.5, iou=0.6)
    r = results[0]
    im_array = r.plot()  # plot a BGR numpy array of predictions

    # Save the image with predictions
    save_path = f'/kaggle/working/{image}'
    cv2.imwrite(save_path, im_array)

    # Display the image
    display(Image.open(save_path))

## 6. Demo

An application is built using the above model. You can visit it in [![Hugging Face Spaces](https://img.shields.io/badge/🤗%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection).


[![Screenshot 2024-07-15 at 12.13.14 PM.jpg](attachment:f857fd59-bdfd-410b-9a87-f3fc74a658ac.jpg)](https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection)


[![Screenshot 2024-07-15 at 12.15.22 PM.png](attachment:036d1a35-42ae-44de-a0ad-1f06d80d8911.png)](https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection)




## 7. Export Ultralytics YOLO Models for mobile or embedded devices.

We need to export the models as `.tflite` and `.mlmodel` files to ensure they are compatible with mobile and embedded devices. To achieve this, we will use the Ultralytics YOLO CLI for model export.

In [ ]:
# Android
!yolo export format=tflite model="/kaggle/working/runs/detect/train/weights/best.pt" imgsz=320 int8 # For detection

# !yolo export format=tflite model=model-cls imgsz=320 int8 # For classification

In [ ]:
# iOS
!yolo export format=mlmodel model="/kaggle/working/runs/detect/train/weights/best.pt" imgsz=320,192 half nms

The converted models are passed to flutter app for detection purpose. Source code can be found [here](https://github.com/sumn2u/yolo-flutter-app/tree/main/example).

[![tomato_ripness_detection.jpg](attachment:ca0d8490-7bd4-440c-bc7c-59dbf24d430c.jpg)](https://github.com/sumn2u/yolo-flutter-app/tree/main/example)


## 8. Conclusion

In this way, we can use YOLO to detect ripe and unripe tomatoes. Thanks to annotation-lab for providing the tools for annotation and dataset.